<a href="https://colab.research.google.com/github/Nihadkaipalli/Predictive-Modeling-for-Asteroid-and-Interstellar-Object-Collision-Risk-Using-Machine-Learning/blob/main/Predictive_Modeling_for_Asteroid_Collision_Risk_Using_Machine_Learning_and_Orbital_Dynamics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install Necessary Libraries**

In [12]:
pip install dash


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3


In [11]:
# Import necessary libraries
# Data Handling
import pandas as pd
import numpy as np

# Preprocessing and Scaling
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Model Training and Evaluation
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Monte Carlo Simulation
from scipy.stats import norm

# Custom Neural Network
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

# Dashboard for Visualization
from dash import Dash, dcc, html
import plotly.express as px

# Email Alerts
import smtplib


ModuleNotFoundError: No module named 'dash'

# **Loading Dataset**

In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Load the dataset from Google Drive
file_path = '/content/drive/My Drive/Asteroid Collision Dataset.csv'
df = pd.read_csv(file_path, low_memory=False)
print("Dataset loaded successfully!")

Dataset loaded successfully!


# **Data Preprocessing**

## Handling missing values

In [9]:
imputer = SimpleImputer(strategy='mean')
df[['H', 'diameter', 'albedo', 'rot_per']] = imputer.fit_transform(df[['H', 'diameter', 'albedo', 'rot_per']])


## Normalizing features

In [10]:
scaler = StandardScaler()
df[['e', 'a', 'q', 'i', 'om', 'w', 'ma', 'n']] = scaler.fit_transform(df[['e', 'a', 'q', 'i', 'om', 'w', 'ma', 'n']])